# Préparation dataset

On crée d'abord un répertoire data pour y ranger nos datasets.

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Add Tag")
secret_value_1 = user_secrets.get_secret("Secret Key")


In [6]:
!rm -r /kaggle/working/data
!rm -r /kaggle/working/Fruitsflow/
!mkdir /kaggle/working/data
%cd /kaggle/working/data

rm: cannot remove '/kaggle/working/data': No such file or directory
rm: cannot remove '/kaggle/working/Fruitsflow/': No such file or directory
/kaggle/working/data


Je me suis placé dans le répertoire data. Je peux y cloner le repo du dataset fruits360.

In [7]:
!git clone https://github.com/971FLS/Fruitsflow

Cloning into 'Fruitsflow'...
remote: Enumerating objects: 50153, done.
remote: Total 50153 (delta 0), reused 0 (delta 0), pack-reused 50153
Receiving objects: 100% (50153/50153), 1.42 GiB | 27.51 MiB/s, done.
Resolving deltas: 100% (7435/7435), done.
Updating files: 100% (35103/35103), done.


Je rentre d'abord l'arborescence jusqu'à atteindre les répertoires Test et Training. Puis, je peux uploader des photos à prédire plus tard.

Notre dataset est prêt. Nous avons même de quoi ranger des éventuels datasets complémentaires.

# Installation YOLO v8

Suivi du process indiqué ici : [Ultralytics](https://docs.ultralytics.com/quickstart/#install)

In [4]:
%cd /kaggle/working/

/kaggle/working


In [5]:
# Dependency ultralytics<=8.0.20 is required but found version=8.0.117, to fix: `pip install ultralytics<=8.0.20`


In [6]:
!pip install ultralytics
#!pip install ultralytics<=8.0.20 # Erreur : /bin/bash: =8.0.20: No such file or directory


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.0/612.0 kB 11.8 MB/s eta 0:00:00a 0:00:01


In [7]:
!pip install opencv-python

In [8]:
!pip install pillow

# Téléchargement du dataset
Dataset : https://public.roboflow.com/object-detection/synthetic-fruit/1

# Entrainement du modèle

In [9]:
url = '/kaggle/working/'

In [10]:
from ultralytics import YOLO

# Create a new YOLO model from scratch
model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training)
#model = YOLO('yolov8m.pt')

# Train the model using the 'coco128.yaml' dataset for 3 epochs
results = model.train(data='/kaggle/working/data/Fruitsflow/dataset/data.yaml', epochs=50, imgsz=320, optimizer='Adam', lr0=0.0001, cos_lr=True)

# Evaluate the model's performance on the validation set
results = model.val()

# Perform object detection on an image using the model
results = model('/kaggle/input/test2206/YOLOv8_multifruits1.jpg')

# Export the model to ONNX format
success = model.export(format='onnx')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
100%|██████████| 6.23M/6.23M [00:00<00:00, 70.6MB/s]
AMP: checks passed ✅
train: Scanning /kaggle/working/data/Fruitsflow/dataset/train/labels... 12285 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12285/12285 [00:08<00:00, 1387.39it/s]
train: New cache created: /kaggle/working/data/Fruitsflow/dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /kaggle/working/data/Fruitsflow/dataset/val/labels... 1755 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1755/1755 [00:01<00:00, 1008.90it/s]
val: New cache created: /kaggle/working/data/Fruitsflow/dataset/val/labels.cache
Plotting labels to runs/detect/train/labels.jpg... 
optimizer: Adam(lr=0.0001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 tr

[Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/numpy-1.24.3.dist-info/METADATA'


ONNX: export success ✅ 1.0s, saved as runs/detect/train/weights/best.onnx (12.7 MB)

Export complete (1.5s)
Results saved to /kaggle/working/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=runs/detect/train/weights/best.onnx imgsz=640 data=/kaggle/working/data/Fruitsflow/dataset/data.yaml 
Visualize:       https://netron.app


================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



## Evaluation du modèle (Val)
Doc : [https://docs.ultralytics.com/modes/val/](https://docs.ultralytics.com/modes/val/)

In [11]:
!pip install ultralytics
!pip install opencv-python
!pip install pillow

In [ ]:
url = "/kaggle/working/"

In [9]:
!python yolo val task=detect model=/kaggle/input/besofl/best.pt imgsz=320 data=/kaggle/working/data/Fruitsflow/dataset/data.yaml

python: can't open file '/kaggle/working/data/yolo': [Errno 2] No such file or directory


In [10]:
# Epoch 1 à 11h57
# Epoch 5 à 14h08
# Epoch 6 à 14h41
# Epoch 9 à 16h18

from ultralytics import YOLO

# Load a model
#model = YOLO('yolov8n.pt')  # load an official model
model = YOLO('/kaggle/input/besofl/best.pt')  # load a custom model



# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.0.122 🚀 Python-3.10.10 torch-2.0.0+cpu CPU
YOLOv8n summary (fused): 168 layers, 3680825 parameters, 0 gradients
100%|██████████| 755k/755k [00:00<00:00, 25.6MB/s]
val: Scanning /kaggle/working/data/Fruitsflow/dataset/val/labels... 1755 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1755/1755 [00:01<00:00, 1444.97it/s]
val: New cache created: /kaggle/working/data/Fruitsflow/dataset/val/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 110/110 [04:46<00:00,  2.61s/it]
                   all       1755       2912      0.999      0.996      0.995      0.994
        Apple Braeburn       1755       1755          1      0.998      0.995      0.994
                 Cocos       1755        119          1      0.994      0.995      0.993
                 Guava       1755        106      0.998          1      0.995      0.995
           Huckleberry       1755        127      0.998          1      0.995   

array([    0.99402,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,
           0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99344,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,
           0.99359,     0.99359,     0.99359,     0.99359,     0.99496,     0.99359,     0.99485,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,     0.99359,
           0.99359,     0.99359,     0.9935

## Prédiction du modèle

In [ ]:
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import cv2


In [14]:
model.predict('/kaggle/input/test2206/YOLOv8_multifruits2.jpg', save=True, imgsz=320, conf=0.5, show=True, save_conf=True, save_crop=True, boxes=True)

error: OpenCV(4.7.0) /io/opencv/modules/highgui/src/window.cpp:1255: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvNamedWindow'


In [25]:
!pip install ultralytics 
from ultralytics import YOLO

# Load a model
#model = YOLO('yolov8n.pt')  # load an official model
model = YOLO('/kaggle/input/besofl/best.pt')  # load a custom model


In [29]:
img = '/kaggle/input/test2206/321_100.jpg'
#img2 = '/kaggle/working/data/Fruitsflow/dataset/val/images/6371.png'
img3 = '/kaggle/input/test2206/1.jpg' # OK 1 Apple Golden 3, 1 Pineapple Mini
img4 = '/kaggle/input/test2206/2.jpg'
img5 = '/kaggle/input/test2206/YOLOv8_multifruits1.jpg'
img6 = '/kaggle/input/test2206/legumes-plantravail.jpg'
img7 = '/kaggle/input/test2206/legumes-SANSplantravail.jpg'
img8 = 'https://www.youtube.com/watch?v=ASFWKTzIt_k'
img8 = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/86/L%C3%A9gumes_pour_ratatouille_au_march%C3%A9_d%27Apt.jpg/2560px-L%C3%A9gumes_pour_ratatouille_au_march%C3%A9_d%27Apt.jpg"
img9 = 'https://fr.wikipedia.org/wiki/Fichier:Avocado_Hass_-_single_and_halved.jpg'
results = model(img9)
boxes = results[0].boxes
box = boxes  # returns one box
box.xyxy


111kB [00:00, 9.35MB/s]


FileNotFoundError: Image Not Found /kaggle/working/data/Fichier:Avocado_Hass_-_single_and_halved.jpg

In [ ]:
import pandas as pd

results.show()
#pandasbox=results.pandas().xyxy[0]
#print(pandasbox)

In [12]:
!zip -r /kaggle/working/220323train30epochsdataset3new3Scratch.zip /kaggle/working/runs/detect/train


  adding: kaggle/working/runs/detect/train/ (stored 0%)
  adding: kaggle/working/runs/detect/train/PR_curve.png (deflated 22%)
  adding: kaggle/working/runs/detect/train/F1_curve.png (deflated 14%)
  adding: kaggle/working/runs/detect/train/val_batch1_labels.jpg (deflated 26%)
  adding: kaggle/working/runs/detect/train/train_batch0.jpg (deflated 20%)
  adding: kaggle/working/runs/detect/train/val_batch2_labels.jpg (deflated 28%)
  adding: kaggle/working/runs/detect/train/val_batch0_pred.jpg (deflated 28%)
  adding: kaggle/working/runs/detect/train/results.png (deflated 10%)
  adding: kaggle/working/runs/detect/train/args.yaml (deflated 51%)
  adding: kaggle/working/runs/detect/train/results.csv (deflated 85%)
  adding: kaggle/working/runs/detect/train/events.out.tfevents.1687657102.7887922fdec0.28.0 (deflated 74%)
  adding: kaggle/working/runs/detect/train/P_curve.png (deflated 19%)
  adding: kaggle/working/runs/detect/train/weights/ (stored 0%)
  adding: kaggle/working/runs/detect/tra

In [31]:
import numpy as np
#from matplotlib import pyplot as plt
%matplotlib inline as plt

plt.imshow(np.squeeze(results.render()))
plt.show()

UsageError: unrecognized arguments: as plt
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.


In [ ]:
!zip -r /kaggle/working/220322_Dataset2new_crops.zip /kaggle/working/runs/detect/predict

In [ ]:
label_folder = '/kaggle/working/data/Fruitsflow/test/labels'
raw_images_folder = '/kaggle/working/data/Fruitsflow/test/labels'
save_images_folder = '/kaggle/working/save_images/'  
name_list_path = '/kaggle/working/data/Fruitsflow/data.yaml'  
classes_path = '/kaggle/working/data/Fruitsflow/data.yaml'


In [ ]:
from PIL import Image, ImageDraw, ImageFont

image_path = "/kaggle/working/data/Fruitsflow/dataset/val/images/6371.png"

x = (0.37519290123456794*360)
y = (0.47241512345679015*360)
width = (0.1388888888888889*360)
height = (0.1388888888888889*360)
output_path = '/kaggle/working/result.png'

def draw_box(image_path, x, y, width, height, output_path):
    # Open the image
    image = Image.open(image_path)

    # Create a draw object
    draw = ImageDraw.Draw(image)

    # Define the box coordinates
    upper_left = (x, y)
    lower_right = (x + width, y + height)

    # Draw the box on the image
    draw.rectangle([upper_left, lower_right], outline='red', width=2)

    # Save the modified image
    image.save(output_path)
    
draw_box(image_path, x, y, width, height, output_path)
im.show(output_path)

In [ ]:
108 0.37519290123456794 0.47241512345679015 0.1388888888888889 0.1388888888888889
108 0.5974151234567902 0.5557484567901235 0.1388888888888889 0.1388888888888889
108 0.8529706790123457 0.6668595679012346 0.1388888888888889 0.1388888888888889
108 0.34463734567901233 0.7779706790123457 0.1388888888888889 0.1388888888888889
108 0.6918595679012346 0.8890817901234568 0.1388888888888889 0.1388888888888889
108 0.8474151234567902 1.0001929012345678 0.1388888888888889 0.1388888888888889
108 0.34463734567901233 0.6668595679012346 0.1388888888888889 0.1388888888888889
108 0.6918595679012346 0.5557484567901235 0.1388888888888889 0.1388888888888889
108 0.8474151234567902 0.47241512345679015 0.1388888888888889 0.1388888888888889
108 0.5974151234567902 0.47241512345679015 0.1388888888888889 0.1388888888888889
108 0.8474151234567902 0.5557484567901235 0.1388888888888889 0.1388888888888889
108 0.6918595679012346 0.6668595679012346 0.1388888888888889 0.1388888888888889

# @chakib : Intégration

In [ ]:
# Chargement du modèle
!pip install ultralytics
!pip install opencv-python
!pip install pillow

In [138]:
# Load le modèle
model = YOLO('/kaggle/input/besofl/best.pt')  # load a custom model

In [139]:
results = model.predict('/kaggle/input/test2206/YOLOv8_multifruits2.jpg', save=True, imgsz=320, conf=0.3, show=True, save_conf=True, save_crop=True, boxes=True)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


image 1/1 /kaggle/input/test2206/YOLOv8_multifruits2.jpg: 256x320 3 Apple Braeburns, 34.4ms
Speed: 1.0ms preprocess, 34.4ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 320)
Results saved to runs/detect/predict5


In [179]:
# Display the annotated frame
#cv2.imshow("YOLOv8 Inference", annotated_frame)
ingredients = []

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    for box in boxes:  # there could be more than one detection
        ingredients.append(int(box.cls))
            
ingredients

[0, 0, 0]

In [121]:
%cd /kaggle/working/
! git clone https://github.com/971FLS/fruits360

/kaggle/working
fatal: destination path 'fruits360' already exists and is not an empty directory.


In [163]:
# Liste des répertoires

import pandas as pd
import os

df=[]
rootdir = '/kaggle/working/fruits360/dataset/Training'
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        df.append(d)

df = pd.DataFrame(data=df, columns=['urls'])
df = df.sort_values(by='urls').reset_index()
df.head()

,index,urls
0,45,/kaggle/working/fruits360/dataset/Training/App...
1,98,/kaggle/working/fruits360/dataset/Training/App...
2,40,/kaggle/working/fruits360/dataset/Training/App...
3,85,/kaggle/working/fruits360/dataset/Training/App...
4,93,/kaggle/working/fruits360/dataset/Training/App...


In [169]:
df['classe'] = df.urls.apply(lambda x: x.split('/')[-1])
df['classe']

0          Apple Braeburn
1      Apple Crimson Snow
2          Apple Golden 1
3          Apple Golden 2
4          Apple Golden 3
              ...        
126         Tomato Maroon
127         Tomato Yellow
128    Tomato not Ripened
129                Walnut
130            Watermelon
Name: classe, Length: 131, dtype: object

In [184]:
df['classe'][0]

'Apple Braeburn'

In [185]:
import pathlib

#url = '/content/gdrive/MyDrive/Final/fruits360/dataset/Training'
url = '/kaggle/working/fruits360/dataset/Training'

ingredients_liste = []
indexes = df.index
classes = df['classe']

for ingredient in ingredients:
    ingredients_liste.append(df['classe'][0])

ingredients_liste

['Apple Braeburn', 'Apple Braeburn', 'Apple Braeburn']